<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# Deployment of a model to Azure App Service and setting CORS policies

## Table of contents

1. [Introduction](#intro)
1. [Model retrieval and export](#model)
1. [Model deployment on Azure](#deploy)
  1. [Workspace retrieval](#workspace)
  1. [Model registration](#register)
  1. [Scoring script](#scoring)
  1. [Environment setup](#env)
  1. [Computational resources](#compute)
  1. [Web service deployment](#websvc)
1. [Notes on web service deployment](#notes)
1. [Clean-up](#clean)
1. [Next steps](#next-steps)

## 1. Introduction <a id="intro"/>

Azure App Service allows users to build and host web apps, mobile back ends, and RESTful APIs in the programming language of one's choice without managing infrastructure. It offers auto-scaling and high availability with support for both Windows and Linux. It also allows for automated deployments from Github, Azure DevOps, or any Git repo. 

For our purposes, we will be using its support for hosting RESTful APIs to deploy our trained machine learning model to a RESTful API endpoint.

In this tutorial, we will learn how to target Azure Apps as a deployment target using the trained model and resources from our previous notebook [21_deployment_on_azure_instances.ipynb](21_deployment_on_azure_instances.ipynb). 

We will also learn how to set Cross-Origin Resource Sharing (CORS) policies that is required when developing a web application hosted in a different domain from the one hosting the deployed machine learning model's RESTful APIs.

To learn more about Cross-Origin Resource Sharing (CORS) policies, go to the [CORS section](#CORS) of this notebook.

### Pre-requisites <a id="pre-reqs"/>

This notebook relies on resources we created in [21_deployment_on_azure_container_instances.ipynb](21_deployment_on_azure_container_instances.ipynb):
- Our Azure Machine Learning workspace
- The Docker image that contains the model and scoring script needed for the web service to work.

If we are missing any of these, we should go back and run the steps from the sections "Pre-requisites" to "3.D Environment setup" to generate them.



## Cross-Origin Resource Sharing (CORS)<a id="CORS"/>

Cross-Origin Resource Sharing (CORS) is an HTTP feature that allows a web application running under a single domain to access resources from another domain. 

By default, web browsers enable [same origin policy](https://www.w3.org/Security/wiki/Same_Origin_Policy), a security mechanism to prevent webpages from calling APIs from a different domain than the one it is calling from. Under the same origin policy, one origin is allowed to send information to another origin, but is not permitted to receive information from another origin to prevent malicious websites from accessing confidential information from other websites.

CORS provides a secure way to allow one origin to both call and receive information from APIs that are hosted in a different origin.

**Hence, setting the correct CORS policy is required if you are building a web application that is hosted in a different domain from the one hosting the deployed machine learning model's RESTful APIs.**

To learn more about CORS in detail, please refer to either the [CORS specification from World Wide Web Consortium (W3C)](https://www.w3.org/TR/cors/) or [Mozilla's documentation page for developers](https://developer.mozilla.org/en-US/docs/Web/HTTP/CORS)